# Generating the metadata file 

In [1]:
#constructing the list with well names
label = []
letras = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] #letters of the rows for a 96-well plate

for linha in letras: #iterate through letters
    for col in range(1, 13): #create a range of 12 columns
        text = str(linha)+'{:01d}'.format(col) #add the letter + the number formated to one numeral, corresponding to Cytation5 nomenclature
        label.append(text) #append text to the label list
#         print(linha,col)
print(label) 

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12']


# Open Excel file with layout

When performing an assay, you should prepare a layout to identificate each well in the plate, and what are the treatments in each well (example below).

Create an excel file with the name of each well in its positions, remembering to keep BLANK the wells were there aren't any treatment.

![alt text](layout.png "Layout")

### Provide the name of your file

In [35]:
filename = input('Whats the name of your file?')

Whats the name of your file? D:\LowPainting\DILI\layout


### Open file and store into layout variable

In [36]:
import openpyxl
from pathlib import Path
#the layout should be in the same directory of the notebook. 
#otherwise, you need to specify the path as Path('Documents', '...xlsx')
set_path = Path(filename+'.xlsx')
#read the excel file
reading = openpyxl.load_workbook(set_path)
#activate the sheet, so you can access other sheets in the excel file
layout = reading.active

### Create a list of names based on excel file

Commands explanation available at https://openpyxl.readthedocs.io/en/latest/api/openpyxl.worksheet.worksheet.html

In [37]:
names = []
for row in layout.iter_rows(max_row=8,max_col=12): #iterate through rows until 8 and cols till 12 (size of the 96 well plate)
    for cell in row: #iterate within the columns in each row
        names.append(str(cell.internal_value)) #append the cell values to the names list

### Confirm the lenght of the list, and print list to confirm the names.

In [38]:
len(names)

96

In [39]:
print(names)

['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'Huh7 Ctrl 0', 'Huh7 Aspirin 1', 'Huh7 Aspirin 10', 'Huh7 Amiodarone 0.2', 'Huh7 Amiodarone 2', 'Huh7 Cyclophosphamide 1', 'Huh7 Cyclophosphamide 10', 'Huh7 Etoposide 1', 'Huh7 Etoposide 10', 'Huh7 Vehicle-ETP 0 ', 'None', 'None', 'Huh7 Ctrl 0', 'Huh7 Aspirin 1', 'Huh7 Aspirin 10', 'Huh7 Amiodarone 0.2', 'Huh7 Amiodarone 2', 'Huh7 Cyclophosphamide 1', 'Huh7 Cyclophosphamide 10', 'Huh7 Etoposide 1', 'Huh7 Etoposide 10', 'Huh7 Vehicle-ETP 0 ', 'None', 'None', 'Huh7 Ctrl 0', 'Huh7 Aspirin 1', 'Huh7 Aspirin 10', 'Huh7 Amiodarone 0.2', 'Huh7 Amiodarone 2', 'Huh7 Cyclophosphamide 1', 'Huh7 Cyclophosphamide 10', 'Huh7 Etoposide 1', 'Huh7 Etoposide 10', 'Huh7 Vehicle-ETP 0 ', 'None', 'None', 'Huh7 Ctrl 0', 'Huh7 Lovastatin 1', 'Huh7 Lovastatin 10', 'Huh7 Orphenadrine 1', 'Huh7 Orphenadrine 10', 'Huh7 Tetracycline 1', 'Huh7 Tetracycline 10', 'Huh7 DMSO 1%', 'Huh7 Lactose 1', 'Huh7 Lactose 10

# Get the name of cell in position 0, time in position 1 and treatment position 2

In [40]:
cells = []
treatment = []
times = []
for x in range(len(names)):
    string = names[x].split()
    if len(string) == 1:
        cells.append('')
        times.append('')
        treatment.append('')
    if len(string) != 1: #exclude the cells that has only 1 string like the None cells
        cells.append(string[0])
        times.append(string[1])
        treatment.append(string[2])

In [41]:
print(treatment)
print(cells)
print(times)

['', '', '', '', '', '', '', '', '', '', '', '', '', '0', '1', '10', '0.2', '2', '1', '10', '1', '10', '0', '', '', '0', '1', '10', '0.2', '2', '1', '10', '1', '10', '0', '', '', '0', '1', '10', '0.2', '2', '1', '10', '1', '10', '0', '', '', '0', '1', '10', '1', '10', '1', '10', '1%', '1', '10', '', '', '0', '1', '10', '1', '10', '1', '10', '1%', '1', '10', '', '', '0', '1', '10', '1', '10', '1', '10', '1%', '1', '10', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', '', '', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', '', '', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', '', '', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', '', '', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', '', '', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 

In [42]:
print(len(treatment))
print(len(cells))
print(len(times))

96
96
96


### Get the name of the cell type (available, in this case, as the first part of the string name) and treatments separately

In [13]:
cells = []
treatment = []
for x in range(len(names)):
    cell = names[x].split()[0] #get the part of the string only at index 0
    cells.append(cell)
    trt = " ".join(names[x].split()[1:]) #get the treatment names separately from the cell's name # 1: is to get everything from index 1
    treatment.append(trt)

In [14]:
print(treatment)

['', '', '', '', '', '', '', '', '', '', '', '', '', '30 Ctrl', '30 Ctrl', '30 Ctrl', '30 Ctrl', '30 Ctrl', '1 Ctrl', '1 Ctrl', '1 Ctrl', '1 Ctrl', '1 Ctrl', '', '', '30 Ag40', '30 Ag40', '30 Ag40', '30 Ag40', '30 Ag40', '1 Ag40', '1 Ag40', '1 Ag40', '1 Ag40', '1 Ag40', '', '', '30 Ag100', '30 Ag100', '30 Ag100', '30 Ag100', '30 Ag100', '1 Ag100', '1 Ag100', '1 Ag100', '1 Ag100', '1 Ag100', '', '', '15 Ctrl', '15 Ctrl', '15 Ctrl', '15 Ctrl', '15 Ctrl', '', '', '', '', '', '', '', '15 Ag40', '15 Ag40', '15 Ag40', '15 Ag40', '15 Ag40', '', '', '', '', '', '', '', '15 Ag100', '15 Ag100', '15 Ag100', '15 Ag100', '15 Ag100', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [79]:
print(cells)

['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'None', 'None', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'None', 'None', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'None', 'None', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'None', 'None', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'None', 'None', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'Huh7', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']


### Use only when z-factor will be calculated within CellProfiler analysis

Create a list to attribute values for each well associated with its biological value.

Used after in cellprofiler for statistical analysis of HCS.

In [38]:
z_factor = []
for n in names:
    if 'Non-treated' in n: #Non-treated cells has a value of 1
        z_factor.append(1)
    elif 'DMSO' in n:
        z_factor.append(-1) #Positive control for damage is equal to -1
    else:
        z_factor.append(0) #Treatments in between has a value of 0
print(z_factor)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Create a list to attribute True when a value is present, and False when None

This step will be useful to create a list of wells and treatments only in the wells with information, and exclude the None wells.

In [43]:
enable = [] #empty list
for name in names: #iterate through the names list
    if name == 'None':
        enable.append(False)
    else:
        enable.append(True)
print(enable)
print(len(enable))

[False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
96


### Concatenate list of names, wells, and z_factor into a single file

### Provide a list of headers to name each column

Run the cell, insert the names of each 

In [44]:
header = [
        'Well',
         'Treatment',
         'Cell',
        'Time',
#         'Z_Factor'
         ]
print(header)

['Well', 'Treatment', 'Cell', 'Time']


In [45]:
filename_out = input() #name of output file

 DILI


In [46]:
#concatenate and save into csv file
import csv

with open('Metadata_' + filename_out + '.csv','w',newline = '') as csvfile: #create a metadata.csv file 
    writer = csv.writer(csvfile, delimiter=',') #comma delimited
#gives the header name row into csv
    writer.writerow([g for g in header])
#data add in csv file
    for j in range(len(enable)):
        if enable[j] == True: #only saves in the csv if there is a value (True)
            wells = label[j]
            treatments = treatment[j]
            cell = cells[j]
            time = times[j]
#             zfactor = z_factor[j]
#if the name of your treatment is already right in the worksheet and there is no need for editing, uncomment the next line
#             treatments = names[j]
            writer.writerow([ #write every line into the csv
                wells,
                treatments,
                cell,
                time,
#                 zfactor
                ])